# BASELINE HISTORICAL DATA

In [ ]:
# Import dependencies
import pandas as pd
import numpy as np
import os
import mysql.connector

from sqlalchemy import create_engine
from config import hostname, port, username, password, connection_string

# Install SQL Magic
!pip install ipython-sql

# Loading  SQL module
%load_ext sql

# Configure SQL Magic to return queries as dataframes
%config SqlMagic.autopandas=True

# Connect to MySQL
cnx = mysql.connector.connect(host = hostname,
                              port = port,
                              user = username,
                              passwd = password,
                              database = "mel_db_test")


# engine = create_engine(connection_string, echo = True)



In [ ]:
# Get list of all database tables
cursor=cnx.cursor()
cursor.execute("SHOW TABLES")
for table_name in cursor:
   print(table_name)

# Read one table into Pandas dataframe
rwa_2018_001 = pd.read_sql_query('Select * FROM Rwanda_2018_BGS_LivelihoodProgram_Baseline', cnx)
rwa_2018_001.head()

# close connection to database
cursor.close()
cnx.close()

column_names = list(rwa_2018_001.columns.values)
column_names

# Add columns with survey ID, survey name
rwa_2018_001.insert(0, 'survey_id', 'RWA-2018-001')
rwa_2018_001.insert(1, 'table_name', 'Rwanda_2018_BGS_LivelihoodProgram_Baseline')
rwa_2018_001

## Removing Irrelevant Data

In [ ]:
# Filter out inactive businesses
filtered_survey_list = ["Kenya_2021_Investment_Kakuma_Q3_Endline",
"Kenya_2021_BGS_LivelihoodProgram_Garissa_Cohort1_Endline",
"Kenya_2021_BGS_LivelihoodProgram_Garissa_Cohort1_Baseline",
"Kenya_2021_BGS_LivelihoodProgram_Kakuma_Cohort4_Endline",
"Rwanda_2021_BGS_LivelihoodProgram_FinancialLiteracy_Endline",
"Rwanda_2020_BGS_CoronaAction_Baseline",
"Rwanda_2020_BGS_GIZ_Endline",
"Rwanda_2020_BGS_OPFMasoro_Endline",
"Rwanda_2020_BGS_OPFMasoro_Baseline",
"Kenya_2020_CovidRecoveryGrant_Kakuma_Endline",
"Kenya_2020_CovidRecoveryGrant_Kakuma_Baseline",
"Rwanda_2020_CovidRecoveryGrant_Endline",
"Rwanda_2021_BGS_LivelihoodProgram_Endline",
"Rwanda_2021_BGS_LivelihoodProgram_Baseline",
"Rwanda_2019_BGS_BKUrumuri_Endline",
"Rwanda_2019_BGS_OPFMasoro_Endline",
"Rwanda_2021_BGS_UGHE_Endline",
"Rwanda_2021_BGS_BKUrumuri_Endline",
"Kenya_2020_BGS_LivelihoodProgram_Kakuma_Cohort2_Endline",
"Kenya_2020_BGS_LivelihoodProgram_Kakuma_Cohort2_Baseline",
"Kenya_2020_BGS_LivelihoodProgram_Kakuma_Cohort3_Baseline",
"Rwanda_2018_BGS_LivelihoodProgram_Endline",
"Rwanda_2020_CovidRecoveryGrant_Baseline",
"Rwanda_2020_BGS_LivelihoodProgram_FinancialLiteracy_Endline",
"Rwanda_2021_BGS_HangaAhazaza_Endline",
"Rwanda_2021_BGS_OPFMasoro_Endline",
"Rwanda_2019_BGS_Kepler_Endline",
"Rwanda_2019_BGS_Kepler_Baseline",
"Rwanda_2020_BGS_HangaAhazaza_Endline",
"Rwanda_2020_BGS_HangaAhazaza_Baseline",
"Rwanda_2020_BGS_Kepler_Endline",
"Rwanda_2020_BGS_Miniyouth_Baseline",
"Rwanda_2020_BGS_Miniyouth_Endline",
"Rwanda_2020_BGS_SNV_Endline",
"Kenya_2019_BGS_LivelihoodProgram_Kakuma_Cohort1_Endline",
"Kenya_2020_BGS_LivelihoodProgram_Kakuma_Cohort2_Endline",
"Kenya_2020_BGS_LivelihoodProgram_Kakuma_Cohort3_Endline",
"Kenya_2019_BGS_LivelihoodProgram_Kakuma_Cohort1_Baseline",
"Kenya_2021_Investment_Kakuma_Q2_Endline"]

i = rwa_2018_001.table_name[0]
if i in filtered_survey_list:
    print("data needs to be filtered")
else:
    print("data does not need to be filtered")

# If data needs to be filtered, do that here


# Import list of relevant columns
columns_to_convert = pd.read_csv('dummy_data_to_convert.csv')
columns_to_convert.head()

columns_to_convert['field_list_concat'][1]

# Split field names from string to list
columns_to_convert['field_list_concat'] = columns_to_convert.field_list_concat.apply(lambda x: x.split(','))
field_list = columns_to_convert.field_list_concat[1]
field_list

# Make sure we're using same table name across the two dataframes
survey_name1 = rwa_2018_001.iloc[1]['table_name']
survey_name2 = columns_to_convert.iloc[1]['table_name']
if survey_name1 == survey_name2:
    field_list = columns_to_convert.iloc[1]['field_list_concat']
    print(field_list)
else:
    print('no match')

# Drop unnecessary columns from dataframe by 
rwa_2018_001 = rwa_2018_001[rwa_2018_001.columns.intersection(field_list)]

rwa_2018_001

# Change demographics column names to match data model
# demo_age
# demo_age_category
# demo_gender_id
# demo_refugee_exp_id
# hh_expense_indicator_base
# jobs_created_base
# revenue_base

rwa_2018_001_base = rwa_2018_001.rename(columns = {'demo_gender': 'demo_gender_id', 
                                                   'entrepreneur_strata': 'demo_refugee_exp_id',
                                                   'employees_count_total_currently': 'jobs_created_base',
                                                    'salesrev_avgrevenue_monthly_last_year': 'revenue_base'})

# If survey didn't include questions for any indicator or metric, add columns here
rwa_2018_001_base['demo_age_category'] = ''
rwa_2018_001_base['hh_expense_indicator_base'] = ''

new_column_names = list(rwa_2018_001_base.columns.values)
new_column_names

# # Rearrange column order to match data model

rwa_2018_001_base_ordered = rwa_2018_001_base.loc[:, ['survey_id',
                                                      'table_name',
                                                      'demo_age',
                                                      'demo_age_category',
                                                      'demo_gender_id',
                                                      'demo_refugee_exp_id',
                                                      'hh_expense_indicator_base',
                                                      'jobs_created_base',
                                                      'revenue_base']]

rwa_2018_001_base_ordered.head()

## Standardize Remaining Data

# # Check data types
rwa_2018_001_base_ordered.dtypes

# # Check null values (some columns are OK to have null values)
rwa_2018_001_base_ordered.isna().sum()

# If no data in demo_age_category column, calculate from age

conditions = [
    (rwa_2018_001_base_ordered['demo_age'] < 36),
    (rwa_2018_001_base_ordered['demo_age'] >= 36) & (rwa_2018_001_base_ordered['demo_age'] < 46),
    (rwa_2018_001_base_ordered['demo_age'] >= 46) & (rwa_2018_001_base_ordered['demo_age'] < 56),
    (rwa_2018_001_base_ordered['demo_age'] >= 56)
]

values = ['18-35 years old', '36-45 years old', '45-55 years old', '56+ years old']

rwa_2018_001_base_ordered['demo_age_category'] = np.select(conditions, values)

rwa_2018_001_base_ordered.head()


# Convert male/female to 0/1 if necessary 
conditions = [
    (rwa_2018_001_base_ordered['demo_gender_id'] == 'Male'),
    (rwa_2018_001_base_ordered['demo_gender_id'] == 'Female')
]

values = [0,1]

rwa_2018_001_base_ordered['demo_gender_id'] = np.select(conditions, values)

rwa_2018_001_base_ordered.head()

# Check values used in demo_refugee_exp_id
rwa_2018_001_base_ordered['demo_refugee_exp_id'].value_counts()

# Convert text values on demo_refugee_exp_id as needed

conditions = [
    (rwa_2018_001_base_ordered['demo_refugee_exp_id'] == 'Refugee Client')
]

values = [0]

rwa_2018_001_base_ordered['demo_refugee_exp_id'] = np.select(conditions, values)

rwa_2018_001_base_ordered.head()

## Transform Raw Data If Needed for Power BI Indicators
#### Note that no changes are needed for jobs - single field, no division or other manipulation needed

In [ ]:
# # Calculate household expense indicator
# None for this survey
rwa_2018_001_base_ordered.head()

# Calculate revenue indicator
revenue_divide = columns_to_convert['revenue_divide_by'][1]
if revenue_divide > 1:
    rwa_2018_001_base_ordered['revenue_base']= rwa_2018_001_base_ordered['revenue_base']/revenue_divide
    rwa_2018_001_base_ordered.revenue_base = rwa_2018_001_base_ordered.revenue_base.round()
else:
    rwa_2018_001_base_ordered['revenue_base'] = rwa_2018_001_base_ordered['revenue_base']

rwa_2018_001_base_ordered.head()

## For First Table Only: Save Dataframe with New Name; Other Tables Will Be Appended to This

In [ ]:
# # Use pd.concat() to append, as in: pd.concat([df1, df2]); this assumes that both dataframes have the same number of columns with same names
baseline_2018_df = rwa_2018_001_base_ordered
baseline_2018_df.head()